<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,036 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,304 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,282 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
G

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [8]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import StringType

In [4]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



In [16]:
def convert_str_upper(s):
  res_str = ""
  arr = s.split(" ")
  for x in arr:
    res_str = res_str+x[0:1].upper()+s[1:len(x)]+" "
  return res_str

In [17]:
convert_str_upper("hello")

'Hello '

In [18]:
convertUDF = udf(lambda z: convert_str_upper(z), StringType())

In [25]:
df1 = (df.select("Seqno",
           convertUDF(col("Name")).alias("name")
           ))

In [26]:
df1.show(truncate=False)

+-----+-------------+
|Seqno|name         |
+-----+-------------+
|1    |John John    |
|2    |Tracey Srace |
|3    |Amy Smy san  |
+-----+-------------+



In [21]:
def uppercase(s):
  return s.upper()

In [22]:
uppercaseUDF = udf(lambda z: uppercase(z), StringType())

In [23]:
df2 = (df.withColumn("curatedName",
                     uppercaseUDF(col("Name"))
                     ))

In [24]:
df2.show(truncate=False)

+-----+------------+------------+
|Seqno|Name        |curatedName |
+-----+------------+------------+
|1    |john jones  |JOHN JONES  |
|2    |tracey smith|TRACEY SMITH|
|3    |amy sanders |AMY SANDERS |
+-----+------------+------------+



In [27]:
spark.udf.register("convertUDF", convert_str_upper, StringType())

<function __main__.convert_str_upper(s)>

In [28]:
df.createOrReplaceTempView("NAME_DF")

In [29]:
spark.sql("select Seqno, convertUDF(name) as Name from NAME_DF").show()

+-----+-------------+
|Seqno|         Name|
+-----+-------------+
|    1|  John John  |
|    2|Tracey Srace |
|    3| Amy Smy san |
+-----+-------------+



In [30]:
@udf(returnType=StringType())
def uppercase(s):
  return s.upper()

In [31]:
df3 = (df.withColumn("curated_name_col",
                     uppercase(col("Name"))
                     ))

In [32]:
df3.show(truncate=False)

+-----+------------+----------------+
|Seqno|Name        |curated_name_col|
+-----+------------+----------------+
|1    |john jones  |JOHN JONES      |
|2    |tracey smith|TRACEY SMITH    |
|3    |amy sanders |AMY SANDERS     |
+-----+------------+----------------+



In [33]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df4 = spark.createDataFrame(data=data,schema=columns)
df4.show(truncate=False)
df4.createOrReplaceTempView("NAME_TABLE2")


+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |null        |
+-----+------------+



In [34]:
spark.sql("select Seqno, convertUDF(Name) from NAME_TABLE2").show(truncate=False)

PythonException: ignored

In [35]:
spark.udf.register("_nullsafeUDF", lambda s: convert_str_upper(s) if not s is None else "", StringType())

<function __main__.<lambda>(s)>

In [37]:
spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2").show(truncate=False)

+------------------+
|_nullsafeUDF(Name)|
+------------------+
|John John         |
|Tracey Srace      |
|Amy Smy san       |
|                  |
+------------------+



In [38]:
spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)

+-----+-----------+
|Seqno|Name       |
+-----+-----------+
|1    |John John  |
+-----+-----------+

